<a href="https://colab.research.google.com/github/fox2056/data-science-bootcamp/blob/main/univariate_time_series_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.26.3

In [2]:
import numpy as np
import pandas as pd

In [3]:
series = pd.read_csv(
    '/content/SolarTimeSeriesData.csv',
    parse_dates =['Datetime'],
    index_col='Datetime')['Incoming Solar']

In [4]:
series[9:20]

,Incoming Solar
Datetime,
2007-10-01 09:00:00,35.4
2007-10-01 10:00:00,63.8
2007-10-01 11:00:00,99.4
2007-10-01 12:00:00,174.5
2007-10-01 13:00:00,157.9
2007-10-01 14:00:00,345.8
2007-10-01 15:00:00,329.8
2007-10-01 16:00:00,114.6
2007-10-01 17:00:00,29.9


In [5]:
series.shift(1)

,Incoming Solar
Datetime,
2007-10-01 00:00:00,NaN
2007-10-01 01:00:00,0.0
2007-10-01 02:00:00,0.0
2007-10-01 03:00:00,0.0
2007-10-01 04:00:00,0.0
...,...
2013-09-30 19:00:00,1.7
2013-09-30 20:00:00,0.0
2013-09-30 21:00:00,0.0


In [6]:
m = 12
series.shift(m)

,Incoming Solar
Datetime,
2007-10-01 00:00:00,NaN
2007-10-01 01:00:00,NaN
2007-10-01 02:00:00,NaN
2007-10-01 03:00:00,NaN
2007-10-01 04:00:00,NaN
...,...
2013-09-30 19:00:00,17.6
2013-09-30 20:00:00,92.0
2013-09-30 21:00:00,163.3


In [7]:
series.expanding().mean()

,Incoming Solar
Datetime,
2007-10-01 00:00:00,0.000000
2007-10-01 01:00:00,0.000000
2007-10-01 02:00:00,0.000000
2007-10-01 03:00:00,0.000000
2007-10-01 04:00:00,0.000000
...,...
2013-09-30 19:00:00,149.795183
2013-09-30 20:00:00,149.792335
2013-09-30 21:00:00,149.789488


In [8]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(series, order=(1,1,1), freq='H')
model_fit = model.fit()
forecasts = model_fit.predict(start=0, end=5, typ='levels')

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:595: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = to_offset(freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)


In [9]:
type(forecasts)

pandas.core.series.Series

In [10]:
forecasts

,predicted_mean
2007-10-01 00:00:00,0.0
2007-10-01 01:00:00,0.0
2007-10-01 02:00:00,0.0
2007-10-01 03:00:00,0.0
2007-10-01 04:00:00,0.0
2007-10-01 05:00:00,0.0


In [11]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if len(data.shape) == 1 else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()

  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]

  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
    else:
      names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]

  agg = pd.concat(cols, axis=1)
  agg.columns = names

  if dropnan:
    agg.dropna(inplace=True)

  return agg

data = series_to_supervised(series, 3)
print(data[10:20])

                     var1(t-3)  var1(t-2)  var1(t-1)  var1(t)
Datetime                                                     
2007-10-01 13:00:00       63.8       99.4      174.5    157.9
2007-10-01 14:00:00       99.4      174.5      157.9    345.8
2007-10-01 15:00:00      174.5      157.9      345.8    329.8
2007-10-01 16:00:00      157.9      345.8      329.8    114.6
2007-10-01 17:00:00      345.8      329.8      114.6     29.9
2007-10-01 18:00:00      329.8      114.6       29.9     10.9
2007-10-01 19:00:00      114.6       29.9       10.9      0.0
2007-10-01 20:00:00       29.9       10.9        0.0      0.0
2007-10-01 21:00:00       10.9        0.0        0.0      0.0
2007-10-01 22:00:00        0.0        0.0        0.0      0.0


### Univariate forecasting with a feedforward neural network

In [12]:
series = series.resample('D').sum()

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch

scaler = MinMaxScaler(feature_range=(-1, 1))
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train = scaler.fit_transform(train)
test = scaler.transform(test)

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

X_train = torch.from_numpy(X_train).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [14]:
import torch.nn as nn

class FeedForwardNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(FeedForwardNN, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)
    self.activation = nn.ReLU()
  def forward(self, x):
    out = self.activation(self.fc1(x))
    out = self.fc2(out)
    return out

model = FeedForwardNN(input_dim=X_train.shape[1],
hidden_dim=32,
output_dim=1)

In [15]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 200

for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  out = model(X_train).reshape(-1,)
  loss = loss_fn(out, y_train)
  loss.backward()
  optimizer.step()
  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 1.05197012424469
Epoch: 10, Loss: 0.7371256947517395
Epoch: 20, Loss: 0.4962281584739685
Epoch: 30, Loss: 0.3252710700035095
Epoch: 40, Loss: 0.21436089277267456
Epoch: 50, Loss: 0.1503293216228485
Epoch: 60, Loss: 0.11844324320554733
Epoch: 70, Loss: 0.10473445057868958
Epoch: 80, Loss: 0.09870074689388275
Epoch: 90, Loss: 0.09455855935811996
Epoch: 100, Loss: 0.09056111425161362
Epoch: 110, Loss: 0.08677086234092712
Epoch: 120, Loss: 0.083041250705719
Epoch: 130, Loss: 0.07935125380754471
Epoch: 140, Loss: 0.07570845633745193
Epoch: 150, Loss: 0.07214576750993729
Epoch: 160, Loss: 0.06868678331375122
Epoch: 170, Loss: 0.06533050537109375
Epoch: 180, Loss: 0.062120433896780014
Epoch: 190, Loss: 0.059105899184942245


In [16]:
model.eval()
y_pred = model(X_test).reshape(-1,)
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.06001007929444313


In [17]:
X_train = X_train.view([X_train.shape[0], X_train.shape[1], 1])
X_test = X_test.view([X_test.shape[0], X_test.shape[1], 1])

In [18]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])

        return out


model = LSTM(input_dim=1,
             hidden_dim=32,
             output_dim=1,
             num_layers=1)

In [19]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1, )
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.7383361458778381
Epoch: 10, Loss: 0.6016860604286194
Epoch: 20, Loss: 0.4617091715335846
Epoch: 30, Loss: 0.3169509470462799
Epoch: 40, Loss: 0.19005601108074188
Epoch: 50, Loss: 0.13287343084812164
Epoch: 60, Loss: 0.13595227897167206
Epoch: 70, Loss: 0.12684199213981628
Epoch: 80, Loss: 0.12227610498666763
Epoch: 90, Loss: 0.11879591643810272
Epoch: 100, Loss: 0.11514713615179062
Epoch: 110, Loss: 0.11212421953678131
Epoch: 120, Loss: 0.1092500388622284
Epoch: 130, Loss: 0.10668356716632843
Epoch: 140, Loss: 0.10433903336524963
Epoch: 150, Loss: 0.1022111028432846
Epoch: 160, Loss: 0.10025656223297119
Epoch: 170, Loss: 0.09842877089977264
Epoch: 180, Loss: 0.09667393565177917
Epoch: 190, Loss: 0.09493813663721085


In [21]:
model.eval()
y_pred = model(X_test)
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([10521])) that is different to the input size (torch.Size([10521, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 0.37564703822135925


### Univariate forecasting with a GRU

In [22]:
X_train = X_train.view([X_train.shape[0], X_train.shape[1], 1])
X_test = X_test.view([X_test.shape[0], X_test.shape[1], 1])

In [23]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out


model = GRUNet(input_dim=1,
               hidden_dim=32,
               output_dim=1,
               num_layers=1)

In [24]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [25]:
epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1, )
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.894960343837738
Epoch: 10, Loss: 0.6935935020446777
Epoch: 20, Loss: 0.5026469826698303
Epoch: 30, Loss: 0.3217230439186096
Epoch: 40, Loss: 0.17694653570652008
Epoch: 50, Loss: 0.1190042495727539
Epoch: 60, Loss: 0.12290085852146149
Epoch: 70, Loss: 0.11313840001821518
Epoch: 80, Loss: 0.10876589268445969
Epoch: 90, Loss: 0.10563357919454575
Epoch: 100, Loss: 0.10232992470264435
Epoch: 110, Loss: 0.09970999509096146
Epoch: 120, Loss: 0.0972246453166008
Epoch: 130, Loss: 0.09500764310359955
Epoch: 140, Loss: 0.09294906258583069
Epoch: 150, Loss: 0.09102579206228256
Epoch: 160, Loss: 0.08918849378824234
Epoch: 170, Loss: 0.08739951252937317
Epoch: 180, Loss: 0.08562180399894714
Epoch: 190, Loss: 0.08382748812437057


In [26]:
model.eval()
y_pred = model(X_test).reshape(-1, )
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.09121344983577728


### Univariate forecasting with a Stacked LSTM

In [27]:
X_train = X_train.view([X_train.shape[0], X_train.shape[1], 1])
X_test = X_test.view([X_test.shape[0], X_test.shape[1], 1])

In [28]:
class StackedLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=2):
        super(StackedLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


model = StackedLSTM(input_dim=1, hidden_dim=32, output_dim=1, num_layers=2)

In [29]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [30]:
epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1,)
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.5322158932685852
Epoch: 10, Loss: 0.4001336097717285
Epoch: 20, Loss: 0.2815380096435547
Epoch: 30, Loss: 0.20342789590358734
Epoch: 40, Loss: 0.20102721452713013
Epoch: 50, Loss: 0.18127912282943726
Epoch: 60, Loss: 0.1695481687784195
Epoch: 70, Loss: 0.1551777869462967
Epoch: 80, Loss: 0.14221082627773285
Epoch: 90, Loss: 0.13095837831497192
Epoch: 100, Loss: 0.12242501229047775
Epoch: 110, Loss: 0.11685501784086227
Epoch: 120, Loss: 0.11324470490217209
Epoch: 130, Loss: 0.11009349673986435
Epoch: 140, Loss: 0.10650099068880081
Epoch: 150, Loss: 0.10225886106491089
Epoch: 160, Loss: 0.0972350686788559
Epoch: 170, Loss: 0.09115639328956604
Epoch: 180, Loss: 0.08367964625358582
Epoch: 190, Loss: 0.07447908818721771


In [31]:
model.eval()
y_pred = model(X_test).reshape(-1,)
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.07011934369802475


### Combining an LSTM with multiple fully connected layers

In [39]:
class HybridLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1):
        super(HybridLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 50)
        self.fc2 = nn.Linear(50, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = F.relu(self.fc1(out[:, -1, :]))
        out = self.fc2(out)
        return out


model = HybridLSTM(input_dim=1, hidden_dim=32, output_dim=1, num_layers=1)

In [40]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [41]:
epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1, )
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.8335340023040771
Epoch: 10, Loss: 0.651729941368103
Epoch: 20, Loss: 0.4640769064426422
Epoch: 30, Loss: 0.282352089881897
Epoch: 40, Loss: 0.16286718845367432
Epoch: 50, Loss: 0.1632995754480362
Epoch: 60, Loss: 0.14388905465602875
Epoch: 70, Loss: 0.13604144752025604
Epoch: 80, Loss: 0.1262313425540924
Epoch: 90, Loss: 0.1180846095085144
Epoch: 100, Loss: 0.1103154867887497
Epoch: 110, Loss: 0.10406220704317093
Epoch: 120, Loss: 0.09932505339384079
Epoch: 130, Loss: 0.0956396833062172
Epoch: 140, Loss: 0.09228350967168808
Epoch: 150, Loss: 0.08870767802000046
Epoch: 160, Loss: 0.08472743630409241
Epoch: 170, Loss: 0.08026724308729172
Epoch: 180, Loss: 0.07531729340553284
Epoch: 190, Loss: 0.06987054646015167


In [42]:
model.eval()
y_pred = model(X_test).reshape(-1, )
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.07092920690774918


### Univariate forecasting with a CNN

In [35]:
import torch.nn.functional as F

class CNNTimeseries(nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super(CNNTimeseries, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_dim,
                               out_channels=64,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.fc = nn.Linear(in_features=64,
                            out_features=output_dim)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


model = CNNTimeseries(input_dim=3, output_dim=1)

In [36]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [37]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(X_train).reshape(-1, )
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

In [38]:
model.eval()
y_pred = model(X_test).reshape(-1, )
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.02887318842113018


### Handling trend – taking first differences

In [43]:
train, test = train_test_split(series, test_size=0.2, shuffle=False)

In [48]:
train_shifted = train.shift(periods=1)
train_diff = train - train_shifted

test_shifted = test.shift(periods=1)
test_diff = test - test_shifted

In [49]:
scaler = MinMaxScaler(feature_range=(-1, 1))

train_diffnorm = scaler.fit_transform(train_diff.values.reshape(-1, 1))
test_diffnorm = scaler.transform(test_diff.values.reshape(-1, 1))

train_df = series_to_supervised(train_diffnorm, n_in=3, n_out=1).values
test_df = series_to_supervised(test_diffnorm, n_in=3, n_out=1).values

In [50]:
X_train, y_train = train_df[:, :-1], train_df[:, -1]
X_test, y_test = test_df[:, :-1], test_df[:, -1]

X_train = torch.from_numpy(X_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor).view(-1)
y_test = torch.from_numpy(y_test).type(torch.Tensor).view(-1)

X_train = X_train.view([X_train.shape[0], X_train.shape[1], 1])
X_test = X_test.view([X_test.shape[0], X_test.shape[1], 1])

In [51]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out


model = LSTM(input_dim=1, hidden_dim=32, output_dim=1, num_layers=2)

In [52]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [53]:
epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1, )
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.06845655292272568
Epoch: 10, Loss: 0.06249884143471718
Epoch: 20, Loss: 0.0621580071747303
Epoch: 30, Loss: 0.061905767768621445
Epoch: 40, Loss: 0.061434555798769
Epoch: 50, Loss: 0.06094628572463989
Epoch: 60, Loss: 0.060253262519836426
Epoch: 70, Loss: 0.059232186526060104
Epoch: 80, Loss: 0.05777755752205849
Epoch: 90, Loss: 0.05594523623585701
Epoch: 100, Loss: 0.054018810391426086
Epoch: 110, Loss: 0.051720961928367615
Epoch: 120, Loss: 0.04987528920173645
Epoch: 130, Loss: 0.049732256680727005
Epoch: 140, Loss: 0.04945544898509979
Epoch: 150, Loss: 0.04931209981441498
Epoch: 160, Loss: 0.04917936772108078
Epoch: 170, Loss: 0.04905994236469269
Epoch: 180, Loss: 0.048950280994176865
Epoch: 190, Loss: 0.048845067620277405


In [54]:
model.eval()
y_pred = model(X_test).reshape(-1, )

y_diff = scaler.inverse_transform(y_pred.detach().numpy().reshape(-1, 1)).flatten()
y_original = y_diff + test_shifted.values[4:]

np.abs(y_original - test.values[4:]).mean()

856.2413883977101

### strona 69